# Create and Test Access to a Spark cluster

## Create the spark cluster

In [19]:
%%local
import json 

cluster_info_string_list = !aws emr create-cluster --auto-scaling-role EMR_AutoScaling_DefaultRole --applications Name=Spark Name=Livy Name=Hive Name=JupyterEnterpriseGateway --tags 'division=MSD' 'environment=Non-Prod' 'AccountID=eto820816420149' 'application=at-research' 'compliance=' 'account owner=Mark Grandau' 'costcenter=MA5150' 'AccountName=molspec' 'Name=ec2-spark-cluster' 'group=AIG' --ebs-root-volume-size 100 --ec2-attributes '{"KeyName":"spark-cluster-key-pair","InstanceProfile":"EMR_EC2_DefaultRole","ServiceAccessSecurityGroup":"sg-0097f1805ac2a9297","SubnetId":"subnet-e2fcc195","EmrManagedSlaveSecurityGroup":"sg-07e0fc489417c4071","EmrManagedMasterSecurityGroup":"sg-0bd43c1510ce6c544","AdditionalMasterSecurityGroups":["sg-7b798a1c"]}' --service-role EMR_DefaultRole --release-label emr-6.3.0 --name 'spark-cluster' --instance-groups '[{"InstanceCount":1,"EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":32,"VolumeType":"gp2"},"VolumesPerInstance":2}]},"InstanceGroupType":"MASTER","InstanceType":"m5.xlarge","Name":"Master Instance Group"},{"InstanceCount":1,"EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":32,"VolumeType":"gp2"},"VolumesPerInstance":2}]},"InstanceGroupType":"CORE","InstanceType":"m5.xlarge","Name":"Core Instance Group"}]' --configurations '[{"Classification":"livy-conf","Properties":{"livy.server.session.timeout":"2h"}}]' --scale-down-behavior TERMINATE_AT_TASK_COMPLETION --region us-west-2
cluster_info_string = ''.join(cluster_info_string_list)

cluster_info = json.loads(f'{cluster_info_string}')

cluster_info["ClusterId"]

print('Wait for the cluster to finish initailizing (https://us-west-2.console.aws.amazon.com/elasticmapreduce/home?region=us-west-2#cluster-list:)')

'j-3TIAKNVYK1MIK'

## Check if the magic has been configured to access the AWS EMR cluster.  The cluster is the Spark server.

In [2]:
%%local
!cat /etc/sparkmagic/config.json

{
  "kernel_python_credentials": {
    "username": "Livy",
    "password": "",
    "url": "http://ip-172-31-43-17.us-west-2.compute.internal:8998",
    "auth": "None"
  },
  "logging_config": {
    "version": 1,
    "formatters": {
      "magicsFormatter": {
        "format": "%(asctime)s\t%(levelname)s\t%(message)s",
        "datefmt": ""
      }
    },
    "handlers": {
      "magicsHandler": {
        "class": "hdijupyterutils.filehandler.MagicsFileHandler",
        "formatter": "magicsFormatter",
        "home_path": "~/.sparkmagic"
      }
    },
    "loggers": {
      "magicsLogger": {
        "handlers": [
          "magicsHandler"
        ],
        "level": "DEBUG",
        "propagate": 0
      }
    }
  },
  "authenticators": {
    "Kerberos": "sparkmagic.auth.kerberos.Kerberos",
    "None": "sparkmagic.auth.customauth.Authenticator",
    "Basic_Access": "sparkmagic.auth.basic.Basic"
  },
  "wait_for_idle_timeout_seconds": 15,
  "livy_session_startup_timeout_seconds": 60,
  "

## DON'T DO THE FOLLOWING AFTER THE KERNEL RESTART

Your going to want to put into any notebook that needs to connect to the spark server the cell bellow, where you hard code to the name of the cluster.

In [24]:
%%local
!sm-sparkmagic connect --cluster-id {cluster_info["ClusterId"]}

Successfully read emr cluster(j-3TIAKNVYK1MIK) details
SparkMagic config file location: /etc/sparkmagic/config.json
Completed setting up configuration files for SparkMagic to connect to EMR cluster j-3TIAKNVYK1MIK


To complete the setup, follow these steps:
1. Restart the kernel. This is required so that SparkMagic can pickup the generated configuration



## Confirm that you have connection to the Spark server.

In [4]:
%%info